## Example 2: Convert TFHub Models to OpenVINO IR

This notebook shows how to convert TFHub models to OpenVINO IR format.

See list of supported models in TFHub-SupportedModelList

Following are the steps
- Enable Python3.8 and Install Python3.8 Devel packages.
- Install necessary pip packages.
- [Optional] In AWS Sagemaker environment, remove few unused conda envs. This will ensure we have enought disk space.
- Select a model from the supported list - TFHub-SupportedModelList
- Convert TFHub Model to OpenVINO IR
- [Optional] Run OpenVINO benchmark

### Enable Python3.8 and Install Python3.8 Devel packages.

In [ ]:
!sudo amazon-linux-extras enable python3.8
!sudo yum -y install python38 python38-devel

### Install necessary pip packages

In [ ]:
!pip3 install --upgrade pip
!pip3 install -r requirements.txt

### [Optional] Removed few unused conda envs to create free space.

In [ ]:
!bash remove-unused-conda-envs.sh

## Convert TFHub Models to OpenVINO IR

### Provide below details:

`bucket_name` - S3 bucket name with "sagemaker" as part of the name.
Example: sagemaker-ir-creation

`tfhub_model_url` - TfHub model URL. See supported list in TFHub-SupportedModelList

`input_shape` - Input shape of the selected model. See TFHub-SupportedModelList

`precision_type` - Use either FP32 or FP16

#### NOTE: If the bucket does not exist then it will be created.

#### Example:
```
bucket_name = "sagemaker-ir-creation"
tfhub_model_url = "https://tfhub.dev/google/efficientnet/b0/classification/1"
input_shape = "1,224,224,3"
precision_type = "FP32"
```

In [ ]:
bucket_name = "sagemaker-ir-creation"
# See supported list in TFHub-SupportedModelList in the repo for URLs and input shapes
tfhub_model_url = "https://tfhub.dev/google/efficientnet/b0/classification/1"
input_shape = "1,224,224,3"
precision_type = "FP32" # FP16 also supported

#### Create IR params.

In [ ]:
url_arr = tfhub_model_url.split("/")
url_arr_len = len(url_arr)
model_name = "".join([url_arr[url_arr_len - 4], "-", url_arr[url_arr_len - 3]])
output_dir = "".join(["./", model_name.replace('.','-'), "-tfhub-", precision_type])
if isinstance(input_shape, str):
    input_shape = [int(i) for i in input_shape.split(",")]

create_ir_params = {
    "tfhub_model_url": tfhub_model_url,
    "output_dir": output_dir,
    "mo_params": {
        "input_shape": input_shape,
        "data_type": precision_type,
        "model_name": model_name
    },
    "bucket_name": bucket_name
}

### Trigger OpenVINO IR creation and upload to S3.

In [ ]:
from ov_utils import create_ir
import os

In [ ]:
try:
    create_ir(create_ir_params) # This will create IR and upload to S3 bucket.
    print (f"IR files created for model:{model_name} and the same is uploaded in S3:{bucket_name}")
    print (f"IR files are located at : {output_dir}/IR_models/")
except Exception as err:
    print(f"FAILED: Please find the error details below:")
    print(err)

### [Optional] Sample Benchmark with OpenVINO IR

In [ ]:
ov_ir_path = "".join([output_dir,"/IR_models/", precision_type, "/", model_name,".xml"])
benchmark_cmd = "".join(["benchmark_app -m ", ov_ir_path, " -niter 50"])
print(f"OpenVINO Benchmarking CMD:\n {benchmark_cmd} \n")

# Run sample openvino bechmark
!{benchmark_cmd}